Task: Recommend 10 unseen songs for every users
Goal: maximize nDCG
Data: user info
      item info
      user interactions with items(test + train)
      item embeddings
Submit: report.txt + codes.zip + recommendations.tsv
TODO: create train-test-val data splits
      setup nDCG evaluation
      produce & evaluate random recommendations
      produce and evaluate POP recommendations

      


In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity as cosine_similarity
from rec import inter_matr_implicit
from tqdm import tqdm
from typing import Callable, List
from sklearn.metrics import ndcg_score
from rec import svd_decompose, svd_recommend_to_list  #SVD
from rec import recTopK  #ItemKNN
from rec import recTopKPop  #TopPop



In [2]:
def inter_matr_implicit(users: int,
                       items: int,
                       interactions: pd.DataFrame,
                       threshold=1) -> np.ndarray:
    """
    Create an implicit interaction matrix from user-item interactions.
    
    Parameters:
        users: DataFrame containing user information
        items: DataFrame containing item information
        interactions: DataFrame containing user-item interaction data
        threshold: Minimum value for a valid interaction (default: 1)
        
    Returns:
        2D numpy array where rows represent users and columns represent items
    """
    interactions = interactions.copy()

    n_users = len(users.index)
    n_items = len(items.index)
    res = np.zeros([n_users, n_items], dtype=np.int8)

    row = interactions['user_id'].to_numpy()
    col = interactions["item_id"].to_numpy()

    data = interactions['count'].to_numpy()
    data[data < threshold] = 0
    data[data >= threshold] = 1

    res[row, col] = data

    return res

def get_ndcg_score_sk(df_predictions, test_interaction_matrix: np.ndarray, topK=10) -> float:
    """
    Calculate the NDCG score for recommendation predictions.
    
    Parameters:
        df_predictions: DataFrame containing recommendation predictions
        test_interaction_matrix: Ground truth interaction matrix
        topK: Number of top recommendations to evaluate (default: 10)
        
    Returns:
        Average NDCG score across all users
    """
    ndcg_avg = 0
    
    for _, row in df_predictions.iterrows():
        g_truth = test_interaction_matrix[row["user_id"]]

        predicted_scores = np.zeros(len(g_truth),dtype=np.int8)

        predictions = list(map(int, row["recs"].split(",")))[:topK]

        for j, rec in enumerate(predictions):
            predicted_scores[rec] = topK-j

        ndcg_avg += ndcg_score(g_truth.reshape(1, -1), predicted_scores.reshape(1, -1), k=topK)

    return ndcg_avg/len(df_predictions)


In [3]:
def read(dataset, file):
    return pd.read_csv(dataset + '/' + dataset + '.' + file, sep='\t')

# Load User Data
users = read('lfm-challenge', 'user')
print("Users Data Head:")
#print(users.head())

# Load Item Data
items = read('lfm-challenge', 'item')
print("\nItem Data Head:")
#print(items.head())

# Load Training Interactions
train_inters = read('lfm-challenge', 'inter_train')
print("\nTraining Interactions Head:")
#print(train_inters.head())

# Load Testing Interactions
test_inters = read('lfm-challenge', 'inter_test')
print("\nTesting Interactions Head:")
#print(test_inters.head())

# Load Embeddings
embedding = read('lfm-challenge', 'musicnn')
print("\nEmbeddings Head:")
print(embedding.head())

train_interaction_matrix = inter_matr_implicit(users, items, train_inters)
test_interaction_matrix = inter_matr_implicit(users, items, test_inters)

Users Data Head:

Item Data Head:

Training Interactions Head:

Testing Interactions Head:

Embeddings Head:
   item_id         0         1         2         3         4         5  \
0        0  0.221942  0.006455  0.027300  0.091775  0.013135  0.137436   
1        1  0.166340  0.000332  0.018895  0.140315  0.002309  0.111743   
2        2  0.247896  0.003749  0.034527  0.036859  0.008251  0.115214   
3        3  0.229554  0.000968  0.028905  0.027514  0.002186  0.100847   
4        4  0.009760  0.000590  0.008925  0.721381  0.000711  0.073143   

          6         7         8  ...        40        41        42        43  \
0  0.082835  0.275749  0.126342  ...  0.058063  0.014128  0.000574  0.001193   
1  0.102853  0.483104  0.135297  ...  0.191162  0.014372  0.000179  0.000249   
2  0.030934  0.609462  0.058102  ...  0.009187  0.005204  0.000456  0.000602   
3  0.029319  0.564656  0.080171  ...  0.008916  0.004114  0.000110  0.000287   
4  0.454569  0.118651  0.368946  ...  0.008477

In [4]:
for n in [5]:
    train_recs_list = []

    for i in range(len(users)):
        train_rec_i = recTopK(train_interaction_matrix, user = i, top_k = 10, n = n)
        train_recs_list.append(",".join(map(str, train_rec_i)))
        if i % 100 == 0:
            print(i)
        
    user_id_list = np.array([ i for i in range(len(users))])

    pop_train_df = pd.DataFrame({"user_id":user_id_list, "recs":train_recs_list})

    print(n,get_ndcg_score_sk(pop_train_df, test_interaction_matrix,10))

0
100


d:\JKU\LFUGD\Challenge\rec.py:141: RuntimeWarning: invalid value encountered in scalar divide
  score = np.sum(intersection) / np.sum(union)


200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
5 0.1894746573112881
